In [ ]:
import time
time.gmtime()

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [3]:
import pandas as pd
import itertools, collections
import time
import numpy as np

from _code import DeepLogModel
from _code import generator
from _code import seed
from _code import trie

In [4]:
### Parameters
_seed=5
subdir = "dynamic-5"
vocabulary_size = 20
num_patterns = 40
num_new_patterns = 5

In [5]:
seed.seed(_seed)
subdir = subdir

In [6]:
vocabulary_size = vocabulary_size
num_patterns = num_patterns
vocabulary = generator.make_vocabulary(vocabulary_size=vocabulary_size)
patterns = generator.generate_patterns(num_patterns=num_patterns, vocabulary=vocabulary, min_pattern_size=3, max_pattern_size=7)
trie_g = trie.calc_g_value(patterns)
trie_h = trie.calc_h_value(patterns)
print("trie-g:", trie_g)
print("trie-h:", trie_h)
text_train, marks_train = generator.generate_text(patterns, text_size=50000, anomaly_ratio=0.00, vocabulary=vocabulary) 
tests1 = generator.generate_tests(patterns, vocabulary, n=700, text_size = 4, anomaly_ratio=0.1)

trie-g: 17
trie-h: 4


In [7]:
import json, os

def write_inputs1(subdir, vocabulary_size, vocabulary, patterns, text_train, marks_train, tests):
    dir_= os.path.join("..\\inputs", subdir)
    os.makedirs(dir_, exist_ok=True)
    j1={
        "vocabulary_size" : vocabulary_size,
        "vocabulary" : vocabulary,
        "workflows" : patterns
    }
    json.dump(j1, open(os.path.join(dir_, "v-wf.json"), "w"))

    j2 = {"text_train" : text_train, "marks_train" : marks_train}
    json.dump(j2, open(os.path.join(dir_, "train.json"), "w"))

    j3 = {"tests" : tests}
    json.dump(j3, open(os.path.join(dir_, "tests1.json"), "w"))

def read_inputs1(subdir):
    dir_ = os.path.join("..\\inputs", subdir)
    j1 = json.load(open(os.path.join(dir_, "v-wf.json")))
    j2 = json.load(open(os.path.join(dir_, "train.json")))
    j3 = json.load(open(os.path.join(dir_, "tests1.json")))
    
    vocabulary_size = j1["vocabulary_size"]
    vocabulary = j1["vocabulary"]
    patterns = j1["workflows"]
    patterns = [tuple(p) for p in patterns]
    
    text_train = j2["text_train"]
    marks_train = j2["marks_train"]
    
    tests_str_keys = j3["tests"]
    tests = dict()
    for k,v in tests_str_keys.items():
        tests[int(k)] = tuple(v)
    
    
    return vocabulary_size, vocabulary, patterns, text_train, marks_train, tests

write_inputs1(subdir, vocabulary_size, vocabulary, patterns, text_train, marks_train, tests1)

In [8]:
ret_vocabulary_size, ret_vocabulary, ret_patterns, ret_text_train, ret_marks_train, ret_tests1 = read_inputs1(subdir)

assert ret_vocabulary_size == vocabulary_size
assert ret_vocabulary == vocabulary
assert ret_patterns == patterns
assert ret_text_train == text_train
assert ret_marks_train == marks_train
assert ret_tests1 == tests1


In [9]:
deep_log_model = DeepLogModel.DeepLogModel(h=trie_h+1, n=vocabulary_size, vocabulary=vocabulary)
deep_log_model.build(num_lstm_layers=2, lstm_size=64)
deep_log_model.fit(text_train,epochs=1)

In [10]:
b = time.time()
entries = list()
for k, (text_test, text_marks, anomaly) in tests1.items():
    for g in range(0, vocabulary_size+1):
        res = deep_log_model.monitor_session(text_test, text_marks, g=g)
        entry = (k,g,res)
        entries.append(entry)
e = time.time()
print("time:", round(e-b,3), "seconds")

time: 709.437 seconds


In [11]:
df1 = pd.DataFrame(entries, columns = ["i","g","status"])
ddf1 = df1.pivot_table(index="i", columns = "g", values = "status", aggfunc = "sum")
def calc(s):
    c = dict(collections.Counter(s))
    TP = c.get("TP", 0)
    TN = c.get("TN", 0)
    FP = c.get("FP", 0)
    FN = c.get("FN", 0)
    eps = 1e-9
    
    prec = TP / (TP + FP + eps)
    rec = TP / (TP + FN + eps)
    acc = (TP + TN) / (TP + TN + FP + FN + eps)
    f1 = 2*(prec*rec)/(prec+rec+eps)
    
    return {"prec" : prec, "rec" : rec, "acc" : acc, "f1" : f1, "TP" : TP, "TN" : TN, "FP" : FP, "FN" : FN}
    
e=ddf1.apply(calc, axis=0)
e1=pd.DataFrame(list(e.values))

print("measure results for each g-value")
e1

measure results for each g-value


,prec,rec,acc,f1,TP,TN,FP,FN
0,0.250000,1.000000,0.250000,0.400000,175,0,525,0
1,0.250000,1.000000,0.250000,0.400000,175,0,525,0
2,0.250000,1.000000,0.250000,0.400000,175,0,525,0
3,0.256223,1.000000,0.274286,0.407925,175,17,508,0
4,0.268105,0.994286,0.320000,0.422330,174,50,475,1
5,0.273450,0.982857,0.342857,0.427861,172,68,457,3
6,0.287415,0.965714,0.392857,0.442988,169,106,419,6
7,0.304673,0.931429,0.451429,0.459155,163,153,372,12
8,0.320000,0.914286,0.492857,0.474074,160,185,340,15
9,0.329741,0.874286,0.524286,0.478873,153,214,311,22


In [12]:
### finding emprical best g
display(e1.loc[[e1["acc"].argmax()]])
display(e1.loc[[e1["f1"].argmax()]])

,prec,rec,acc,f1,TP,TN,FP,FN
17,1.0,0.205714,0.801429,0.341232,36,525,0,139


,prec,rec,acc,f1,TP,TN,FP,FN
12,0.417508,0.708571,0.68,0.525424,124,352,173,51


In [13]:
best_g = e1["f1"].argmax()
print("best-g", best_g)

best-g 12


## online_cases

In [14]:
num_new_patterns = num_new_patterns
new_patterns = generator.generate_patterns(num_patterns=num_new_patterns, vocabulary=vocabulary, min_pattern_size=3, max_pattern_size=7)

online_cases = generator.generate_tests(patterns + new_patterns, vocabulary, n=10000, text_size = 4, anomaly_ratio=0.0)    

In [15]:
def write_inputs2(subdir, online_cases):
    dir_= os.path.join("..\\inputs", subdir)
    os.makedirs(dir_, exist_ok=True)
    
    j4 = {"online_cases" : online_cases}
    json.dump(j4, open(os.path.join(dir_, "online_cases.json"), "w"))

def read_inputs2(subdir):
    dir_ = os.path.join("..\\inputs", subdir)
    
    j4 = json.load(open(os.path.join(dir_, "online_cases.json")))
    online_cases_str_keys = j4["online_cases"]
    online_cases = dict()
    for k,v in online_cases_str_keys.items():
        online_cases[int(k)] = tuple(v)

    return online_cases

In [16]:
write_inputs2(subdir, online_cases)
ret_online_cases = read_inputs2(subdir)
assert ret_online_cases == online_cases

In [17]:
trie_g_new = trie.calc_g_value(patterns + new_patterns)
trie_h_new = trie.calc_h_value(patterns + new_patterns)
print("best-g:", best_g)
print("trie-g new: {}. (prev trie-g: {})".format(trie_g_new, trie_g))
print("trie-h new: {}. (prev trie-h: {})".format(trie_h_new, trie_h))
best_g_new = max(best_g, trie_g_new)
print("best-g-new:", best_g_new)

best-g: 12
trie-g new: 19. (prev trie-g: 17)
trie-h new: 4. (prev trie-h: 4)
best-g-new: 19


In [18]:
num_feedbacks = 0
num_feedbacks_new = 0
num_feedbacks_old = 0

def contains_new_pattern(text, new_patterns):
    new_ps_str = ["#".join(p) for p in new_patterns]
    text_str = "#".join(text)
    for new_p_str in new_ps_str:
        if new_p_str in text_str:
            return True
    return False
    
for index, case in online_cases.items():
    text, marks, anomaly_exists = case
    res = deep_log_model.monitor_session(text, marks, g=best_g)
    if res == "FP":
        deep_log_model.train_feedback(text, marks, g = best_g) # we are using here the old empirical best g. The theoretical g (trie_g_new) might be higher and give at the end worse results.
        num_feedbacks += 1
        if contains_new_pattern(text, new_patterns):
            num_feedbacks_new +=1
        else:
            num_feedbacks_old +=1 # count only totally "old" feedback cases without any new workflow 
        

In [19]:
print("num_feedbacks_new", num_feedbacks_new)
print("num_feedbacks_old", num_feedbacks_old)
print("num_feedbacks", num_feedbacks)

num_feedbacks_new 2372
num_feedbacks_old 3381
num_feedbacks 5753


In [20]:
tests2 = generator.generate_tests(patterns + new_patterns, vocabulary, n=700, text_size = 4, anomaly_ratio=0.1)

In [21]:
def write_inputs3(subdir, tests):
    dir_= os.path.join("..\\inputs", subdir)
    os.makedirs(dir_, exist_ok=True)
    
    j5 = {"tests" : tests}
    json.dump(j5, open(os.path.join(dir_, "tests2.json"), "w"))

def read_inputs3(subdir):
    dir_= os.path.join("..\\inputs", subdir)
    j5 = json.load(open(os.path.join(dir_, "tests2.json")))
    
    tests_str_keys = j5["tests"]
    tests = dict()
    for k,v in tests_str_keys.items():
        tests[int(k)] = tuple(v)
    return tests

In [22]:
write_inputs3(subdir, tests2)
ret_tests2 = read_inputs3(subdir)
assert ret_tests2 == tests2

In [23]:
b = time.time()
entries = list()
for k, (text_test, text_marks, anomaly) in tests2.items():
    for g in range(best_g, vocabulary_size+1): # adding WF can't cause lower g.
        res = deep_log_model.monitor_session(text_test, text_marks, g=g)
        entry = (k,g,res)
        entries.append(entry)
e = time.time()
print("time:", round(e-b,3), "seconds")

time: 303.443 seconds


In [24]:
df2 = pd.DataFrame(entries, columns = ["i","g","status"])
ddf2 = df2.pivot_table(index="i", columns = "g", values = "status", aggfunc = "sum")
def calc(s):
    c = dict(collections.Counter(s))
    TP = c.get("TP", 0)
    TN = c.get("TN", 0)
    FP = c.get("FP", 0)
    FN = c.get("FN", 0)
    eps = 1e-9
    
    prec = TP / (TP + FP + eps)
    rec = TP / (TP + FN + eps)
    acc = (TP + TN) / (TP + TN + FP + FN + eps)
    f1 = 2*(prec*rec)/(prec+rec+eps)
    
    return {"g" : s.name, "prec" : prec, "rec" : rec, "acc" : acc, "f1" : f1, "TP" : TP, "TN" : TN, "FP" : FP, "FN" : FN}
    
e=ddf2.apply(calc, axis=0)
e2=pd.DataFrame(list(e.values))

print("measure results for each g-value")
e2

measure results for each g-value


,g,prec,rec,acc,f1,TP,TN,FP,FN
0,12,0.352941,0.830769,0.528571,0.495413,162,208,297,33
1,13,0.373171,0.784615,0.572857,0.505785,153,248,257,42
2,14,0.402299,0.717949,0.624286,0.515654,140,297,208,55
3,15,0.418367,0.630769,0.652857,0.503067,123,334,171,72
4,16,0.453390,0.548718,0.690000,0.496520,107,376,129,88
5,17,0.514286,0.461538,0.728571,0.486486,90,420,85,105
6,18,0.622449,0.312821,0.755714,0.416382,61,468,37,134
7,19,1.000000,0.123077,0.755714,0.219178,24,505,0,171
8,20,0.000000,0.000000,0.721429,0.000000,0,505,0,195


In [25]:
best_result2 = e2.loc[[e2["f1"].argmax()]]
best_g2 = best_result2["g"].iloc[0]

In [26]:
e3 = e2[e2["g"] == best_g2].copy()
e3["num_feedbacks_new"] = num_feedbacks_new
e3["num_feedbacks_old"] = num_feedbacks_old
e3["num_feedbacks"] = num_feedbacks
e3

,g,prec,rec,acc,f1,TP,TN,FP,FN,num_feedbacks_new,num_feedbacks_old,num_feedbacks
2,14,0.402299,0.717949,0.624286,0.515654,140,297,208,55,2372,3381,5753
